# Data prep notebook

Aims to collect the raw data, apply transformations for better usage and then deliver a refined dataset ready for analysis

In [2]:
import pandas as pd
import json
pd.set_option('display.max_columns', None)

## raw

In [3]:
df = pd.read_csv("dataset/raw.tsv", sep='\t')

# removing dirt
df = df[df.columns[:-4]] 

## data catalog

In [4]:
# Opening JSON file
with open('catalogo.json') as arquivo:
    catalogo = json.load(arquivo)["schema"]

## processing

In [5]:
nomes = dict()

for item in catalogo:
    # creating new boolean columns based on string contains  
    if "apply_contains" in item:
        if "qt" not in item["column"]:
            df.loc[df[item["survey_question"]].str.contains(item["apply_contains"], na=False), item["column"]] = 1
    
    # creating dict for renaming columns
    elif "qt" not in item["column"]:
        nomes |= {item["survey_question"]:item["column"]}

# drop unecessary columns or the ones which is already processed
df.drop('Marque quais metodologias ágeis foram utilizadas no projeto de desenvolvimento da solução de Data Lake na nuvem em sua empresa', axis=1, inplace=True)
df.drop('Houve alteração de escopo no projeto?', axis=1, inplace=True)
df.drop('Quais foram os tipos de dados solicitados para ingestão?', axis=1, inplace=True)
df.drop('Indique abaixo quais foram as principais falhas que ocorreram no desenvolvimento da solução de Data Lake na nuvem em sua empresa', axis=1, inplace=True)
df.drop('Comentários', axis=1, inplace=True)
df.drop('Qual é o tamanho aproximado do projeto em armazenamento? (Ex resposta: 130GB)',   axis=1, inplace=True)
df.drop('Houve ingestão de streaming de dados?', axis=1, inplace=True)
df.drop('Se você respondeu SIM para a questão anterior, por gentileza, indique abaixo quais alterações ocorreram no escopo do projeto.', axis=1, inplace=True)
df.drop('Carimbo de data/hora', axis=1, inplace=True)

# saying again... renaming
nomes |= {'Projetos de big data possuem 3 desafios principais: Volume, Velocidade, Variedade. Para demonstrar que você está respondendo esse questionário com atenção, por favor, selecione apenas a opção “Velocidade” dentre as opções abaixo: ':'teste_atencao'}
nomes |= {'A empresa que você trabalha está ou esteve envolvida em um projeto para desenvolver uma solução de DATA LAKE?':'trabalhou_com_dl'}
df = df.rename(columns=nomes)

/var/folders/wy/6nbc6l7s7d3bzmvzj3rkssfm0000gp/T/ipykernel_41217/3994124315.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df.loc[df[item["survey_question"]].str.contains(item["apply_contains"], na=False), item["column"]] = 1


In [6]:
df.loc[df['maior_desafio'].str.contains('Segurança', na=False),'maior_desafio'] = 'Variedade'
df.loc[df['maior_desafio'].str.contains('Complexidade', na=False),'maior_desafio'] = 'Velocidade'
df.loc[df['maior_desafio'].str.contains('Disponibilidade', na=False),'maior_desafio'] = 'Velocidade'
df.loc[df['maior_desafio'].str.contains('Recorrência', na=False),'maior_desafio'] = 'Velocidade'

#### simplifying values

In [7]:
## Cloud provider
df.replace(
    to_replace={
        "Amazon Web Services":"AWS",
        "Microsoft Azure":"Azure",
        "Google Cloud Plataform":"GCP"
    }, 
    inplace=True
)


## Porte da empresa
df.replace(
    to_replace={
    "Empresa de Grande Porte (500 funcionários ou mais)":"]500,+∞]",
    "Empresa de Pequeno Porte (até 100 funcionários)":"]100,500]",
    "Empresa de Médio Porte (até 500 funcionários)":"]20,50]",
    "Microempresa (até 20 funcionários)":"[0,20]"
    },
    inplace=True
)

## tamanho do investimento 
df.replace(
    to_replace={
        "Acima de R$ 100.000,01":"]100,+∞]",
        "Não sei responder essa informação":"N/A",
        "Entre R$ 50.000,01 e R$ 100.000,00":"]50,100]",
        "Entre R$ 20.000,01 e R$ 30.000,00":"]20,30]",
        "Entre R$ 10.000,01 e R$ 20.000,00":"]10,20]",
        "Entre R$ 30.000,01 e R$ 50.000,00":"]30,50]",
        "Abaixo de R$ 10.000,00":"[0,10]"
    },
    inplace=True
)

## Tempo do projeto
df.replace(
    to_replace={
        "3 a 6 meses":"]3,6]",
        "6 a 12 meses":"]6,12]",
        "Até 3 meses":"[0,3]",
        "12 a 18 meses":"]12,18]",
        "18 meses ou mais":"]18,+∞]"
    },
    inplace=True
)

# Tamanho da equipe
df.replace(
    to_replace={
        "35 pessoas ou mais":"]35,+∞]",
        "5 a 10 pessoas":"]5,10]",
        "11 a 20 pessoas":"]10,20]",
        "Até 5 pessoas":"[0,5]",
        "21 a 35 pessoas":"]20,35]"
    },
    inplace=True
)

# Segmento
df.replace(
    to_replace={
        "Ambos":"B2ALL",
        "B2B (Empresas negociam com outras empresas)":"B2B",
        "B2C (Empresas negociam com consumidores finais)":"B2C"
    },
    inplace=True
)

# Posição
df.replace(
    to_replace={
        "Colaborador Interno":"Interno",
        "Consultor Terceirizado":"Terceiro"
    },
    inplace=True
)

# Posição
df.replace(
    to_replace={
        "Desenvolvimento":"Dev"
    },
    inplace=True
)

# to int
df["avaliacao"] = df["avaliacao"].fillna(0).astype('int64')

# to int 0 + 1
df["bool_experiencia"] = df["bool_experiencia"].apply(lambda x: 1 if x == "Sim" else 0)
df["bool_cert"] = df["bool_cert"].apply(lambda x: 1 if x == "Sim" else 0)

bools = [col for col in df.columns if "bool" in col]
for b in bools:
    df[b] = df[b].fillna(0).astype('int64')

#### add calculated columns

In [8]:
df["qt_alteracoes"] = df["bool_alt_eqp"] + df["bool_alt_bd"] + df["bool_alt_sol"] + df["bool_alt_custo"] + df["bool_alt_tec"]
df["qt_falhas"] = df["bool_falha_gov"] + df["bool_falha_dev"] + df["bool_falha_aus_info"] + df["bool_falha_pln_custo"] + df["bool_falha_gst_lid"] + df["bool_falha_usu"] 

## filtering valid inputs

##### people who does not have previous contact with data lake projects 

In [9]:
df.groupby(["trabalhou_com_dl"]).size()

trabalhou_com_dl
Não     35
Sim    106
dtype: int64

In [10]:
df = df.query("trabalhou_com_dl == 'Sim'")

##### people who does not pay attention when answering

In [11]:
df.groupby(["teste_atencao"]).size()

teste_atencao
Variedade, Velocidade, Volume      2
Variedade, Volume                  1
Velocidade                       102
Volume                             1
dtype: int64

In [12]:
df = df.query("teste_atencao == 'Velocidade'")

In [13]:
print(f"Size: {len(df.index)}")

Size: 102


##### creating new bool columns dropping unnecessary for analysis

In [14]:
df["bool_terceiro"] = df["atua_como"].map(lambda x: 1 if x == "Terceiro" else 0)
df["bool_dev"] = df["posicao"].map(lambda x: 1 if x == "Dev" else 0)

In [15]:
df.drop(
    [
        "teste_atencao",
        "trabalhou_com_dl",
        "atua_como",
        "posicao"
    ], inplace=True, axis=1)

##### writing new dataset 

In [16]:
df.to_csv("dataset/refined.tsv", sep="\t", index=False)

In [17]:
# Data preparation noteboook